# Sample for Kubeflow TFJob SDK

This is a sample for Kubeflow TFJob SDK `kubeflow-tfjob`.

The notebook shows how to use Kubeflow TFJob SDK to create, get, wait, check and delete tfjob.

In [1]:
from kubernetes.client import V1PodTemplateSpec
from kubernetes.client import V1ObjectMeta
from kubernetes.client import V1PodSpec
from kubernetes.client import V1Container

from kubeflow.training import constants
from kubeflow.training.utils import utils
from kubeflow.training import V1ReplicaSpec
from kubeflow.training import KubeflowOrgV1TFJob
from kubeflow.training import KubeflowOrgV1TFJobSpec
from kubeflow.training import V1RunPolicy
from kubeflow.training import TFJobClient

Define namespace where tfjob needs to be created to. If not specified, below function defines namespace to the current one where SDK is running in the cluster, otherwise it will deploy to default namespace.

In [2]:
namespace = utils.get_default_target_namespace()

### Define TFJob

The demo only creates a worker of TFJob to run mnist sample.

In [3]:
container = V1Container(
    name="tensorflow",
    image="gcr.io/kubeflow-ci/tf-mnist-with-summaries:1.0",
    command=[
        "python",
        "/var/tf_mnist/mnist_with_summaries.py",
        "--log_dir=/train/logs", "--learning_rate=0.01",
        "--batch_size=150"
        ]
)

worker = V1ReplicaSpec(
    replicas=2,
    restart_policy="Never",
    template=V1PodTemplateSpec(
        spec=V1PodSpec(
            containers=[container]
        )
    )
)

chief = V1ReplicaSpec(
    replicas=1,
    restart_policy="Never",
    template=V1PodTemplateSpec(
        spec=V1PodSpec(
            containers=[container]
        )
    )
)

ps = V1ReplicaSpec(
    replicas=1,
    restart_policy="Never",
    template=V1PodTemplateSpec(
        spec=V1PodSpec(
            containers=[container]
        )
    )
)

tfjob = KubeflowOrgV1TFJob(
    api_version="kubeflow.org/v1",
    kind="TFJob",
    metadata=V1ObjectMeta(name="mnist",namespace=namespace),
    spec=KubeflowOrgV1TFJobSpec(
        run_policy=V1RunPolicy(clean_pod_policy="None"),
        tf_replica_specs={"Worker": worker,
                          "Chief": chief,
                          "PS": ps}
    )
)

### Create TFJob

In [4]:
tfjob_client = TFJobClient()
tfjob_client.create(tfjob, namespace=namespace)

{'apiVersion': 'kubeflow.org/v1',
 'kind': 'TFJob',
 'metadata': {'creationTimestamp': '2021-10-02T19:02:08Z',
  'generation': 1,
  'managedFields': [{'apiVersion': 'kubeflow.org/v1',
    'fieldsType': 'FieldsV1',
    'fieldsV1': {'f:spec': {'.': {},
      'f:runPolicy': {'.': {}, 'f:cleanPodPolicy': {}},
      'f:tfReplicaSpecs': {'.': {},
       'f:Chief': {'.': {},
        'f:replicas': {},
        'f:restartPolicy': {},
        'f:template': {'.': {}, 'f:spec': {'.': {}, 'f:containers': {}}}},
       'f:PS': {'.': {},
        'f:replicas': {},
        'f:restartPolicy': {},
        'f:template': {'.': {}, 'f:spec': {'.': {}, 'f:containers': {}}}},
       'f:Worker': {'.': {},
        'f:replicas': {},
        'f:restartPolicy': {},
        'f:template': {'.': {}, 'f:spec': {'.': {}, 'f:containers': {}}}}}}},
    'manager': 'OpenAPI-Generator',
    'operation': 'Update',
    'time': '2021-10-02T19:02:08Z'}],
  'name': 'mnist',
  'namespace': 'default',
  'resourceVersion': '6042',
 

### Get the created TFJob 

In [5]:
tfjob_client.get('mnist', namespace=namespace)

{'apiVersion': 'kubeflow.org/v1',
 'kind': 'TFJob',
 'metadata': {'creationTimestamp': '2021-10-02T19:02:08Z',
  'generation': 1,
  'managedFields': [{'apiVersion': 'kubeflow.org/v1',
    'fieldsType': 'FieldsV1',
    'fieldsV1': {'f:spec': {'.': {},
      'f:runPolicy': {'.': {}, 'f:cleanPodPolicy': {}},
      'f:tfReplicaSpecs': {'.': {},
       'f:Chief': {'.': {},
        'f:replicas': {},
        'f:restartPolicy': {},
        'f:template': {'.': {}, 'f:spec': {'.': {}, 'f:containers': {}}}},
       'f:PS': {'.': {},
        'f:replicas': {},
        'f:restartPolicy': {},
        'f:template': {'.': {}, 'f:spec': {'.': {}, 'f:containers': {}}}},
       'f:Worker': {'.': {},
        'f:replicas': {},
        'f:restartPolicy': {},
        'f:template': {'.': {}, 'f:spec': {'.': {}, 'f:containers': {}}}}}}},
    'manager': 'OpenAPI-Generator',
    'operation': 'Update',
    'time': '2021-10-02T19:02:08Z'},
   {'apiVersion': 'kubeflow.org/v1',
    'fieldsType': 'FieldsV1',
    'fiel

### Get the TFJob status, check if the TFJob has been started.

In [6]:
tfjob_client.get_job_status('mnist', namespace=namespace)

'Running'

### Wait for the specified job to finish

In [7]:
tfjob_client.wait_for_job('mnist', namespace=namespace, watch=True)

NAME                           STATE                TIME                          
mnist                          Running              2021-10-02T19:02:10Z          
mnist                          Running              2021-10-02T19:02:10Z          
mnist                          Running              2021-10-02T19:02:10Z          
mnist                          Succeeded            2021-10-02T19:04:10Z          


### Check if the TFJob succeeded

In [8]:
tfjob_client.is_job_succeeded('mnist', namespace=namespace)

True

### Get the TFJob training logs.

In [9]:
tfjob_client.get_logs('mnist', namespace=namespace)

The logs of Pod mnist-chief-0:
Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.
Instructions for updating:
Please write your own downloading logic.
Instructions for updating:
Please use urllib or similar directly.
Instructions for updating:
Please use tf.data to implement this functionality.
Instructions for updating:
Please use tf.data to implement this functionality.
Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.
2021-10-02 19:02:25.434889: I tensorflow/core/platform/cpu_feature_guard.cc:141] Your CPU supports instructions that this TensorFlow binary was not compiled to use: AVX2 FMA
Successfully downloaded train-images-idx3-ubyte.gz 9912422 bytes.
Extracting /tmp/tensorflow/mnist/input_data/train-images-idx3-ubyte.gz
Successfully downloaded train-labels-idx1-ubyte.gz 28881 bytes.
Extracting /tmp/tensorflow/mnist/input_data/train-labels-idx1-ubyte.gz
Successfully 

### Delete the TFJob

In [10]:
tfjob_client.delete('mnist', namespace=namespace)

{'kind': 'Status',
 'apiVersion': 'v1',
 'metadata': {},
 'status': 'Success',
 'details': {'name': 'mnist',
  'group': 'kubeflow.org',
  'kind': 'tfjobs',
  'uid': '4a0b9764-b5c4-4d30-95c3-d3c56d342803'}}